In [2]:
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from ipywidgets import widgets

In [4]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1)

In [6]:
df.sample(3)

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute
78969,2013,11,25,2117.0,42.0,2237.0,27.0,WN,N704SW,168,LGA,MDW,125.0,725,21.0,17.0
312547,2013,9,4,1753.0,-7.0,1903.0,-12.0,US,N746UW,2191,LGA,DCA,45.0,214,17.0,53.0
199411,2013,5,7,1302.0,-13.0,1610.0,-6.0,B6,N568JB,1601,LGA,RSW,150.0,1080,13.0,2.0


In [8]:
df['carrier'].unique()

array(['UA', 'AA', 'B6', 'DL', 'EV', 'MQ', 'US', 'WN', 'VX', 'FL', 'AS',
       '9E', 'F9', 'HA', 'YV', 'OO'], dtype=object)

In [16]:
month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

container = widgets.HBox(children=[use_date, month])

textbox = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)


# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        ),
                        barmode='overlay'
                    ))

In [18]:
def validate():
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in
                           zip(df['month'] == month.value, df['carrier'] == textbox.value,
                               df['origin'] == origin.value)]
            temp_df = df[filter_list]

        else:
            filter_list = [i and j for i, j in
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'


origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")

In [20]:
container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,
              container2,
              g])

TraitError: The 'children' trait of a VBox instance contains an Instance of a TypedTuple which expected a Widget, not the Figure Figure({
    'data': [{'name': 'Arrival Delays',
              'opacity': 0.75,
              'type': 'histogram',
              'x': {'bdata': ('AAAAAAAAJkAAAAAAAAA0QAAAAAAAgE' ... 'AAAPh/AAAAAAAA+H8AAAAAAAD4fw=='),
                    'dtype': 'f8'}},
             {'name': 'Departure Delays',
              'opacity': 0.75,
              'type': 'histogram',
              'x': {'bdata': ('AAAAAAAAAEAAAAAAAAAQQAAAAAAAAA' ... 'AAAPh/AAAAAAAA+H8AAAAAAAD4fw=='),
                    'dtype': 'f8'}}],
    'layout': {'barmode': 'overlay', 'template': '...', 'title': {'text': 'NYC FlightDatabase'}}
}).